In [1]:
import openai
import pinecone
import os
import huggingface_hub
import pinecone
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
huggingface_api_token = os.getenv('HUGGINGFACE_API_TOKEN')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
openai.api_key = openai_api_key
huggingface_hub.api_token = huggingface_api_token
pinecone.api_key = pinecone_api_key
import openai
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

/workspaces/MCQ-Creator-App/.venv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
def load_docs(directory):
  loader = PyPDFDirectoryLoader(directory)
  documents = loader.load()
  return documents
directory = '.'
documents = load_docs(directory)
len(documents)
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs
docs = split_docs(documents)

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
query_result = embeddings.embed_query("Hello Buddy")
len(query_result)
import pinecone
pinecone.init(
    api_key='',
    environment="gcp-starter"
)

index_name = "mcq-creator"

index = Pinecone.from_documents(docs, embeddings, index_name=index_name)
def get_similiar_docs(query, k=2):
    similar_docs = index.similarity_search(query, k=k)
    return similar_docs
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
llm=HuggingFaceHub(repo_id="bigscience/bloom", model_kwargs={"temperature":1e-10})
llm
chain = load_qa_chain(llm, chain_type="stuff")
def get_answer(query):
  relevant_docs = get_similiar_docs(query)
  print(relevant_docs)
def get_answer(query):
  relevant_docs = get_similiar_docs(query)
  print(relevant_docs)
  response = chain.run(input_documents=relevant_docs, question=query)
  return response
answer= get_answer("How is India's economy?")
import re
import json

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
response_schemas = [
    ResponseSchema(name="question", description="Question generated from provided input text data."),
    ResponseSchema(name="choices", description="Available options for a multiple-choice question in comma separated."),
    ResponseSchema(name="answer", description="Correct answer for the asked question.")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
output_parser
# This helps us fetch the instructions the langchain creates to fetch the response in desired format
format_instructions = output_parser.get_format_instructions()
 
print(format_instructions)
# create ChatGPT object
chat_model = ChatOpenAI()

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("""When a text input is given by the user, please generate multiple choice questions 
        from it along with the correct answer. 
        \n{format_instructions}\n{user_prompt}""")  
    ],
    input_variables=["user_prompt"],
    partial_variables={"format_instructions": format_instructions}
)
final_query = prompt.format_prompt(user_prompt = answer)
print(final_query)
final_query.to_messages()
final_query_output = chat_model(final_query.to_messages())
print(final_query_output.content)

# Let's extract JSON data from Markdown text that we have
markdown_text = final_query_output.content
json_string = re.search(r'{(.*?)}', markdown_text, re.DOTALL).group(1)

In [3]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [4]:
llm = OpenAI()

In [5]:
prompt = PromptTemplate(
    input_variables=["user_prompt"],
    template ="Best places to visit in {user_prompt}",
)

In [6]:
chain = LLMChain(llm=llm, prompt=prompt)

In [8]:
print(chain.run(user_prompt="Singapore"))  



1. Singapore Botanic Gardens: The Singapore Botanic Gardens is a popular tourist attraction and a UNESCO World Heritage Site. It is home to over 10,000 species of plants, including the world's most diverse collection of orchids.

2. Singapore Zoo: Singapore Zoo is one of the world's most respected and popular zoos, home to over 2,500 animals in open, naturalistic habitats.

3. Marina Bay Sands: The Marina Bay Sands is a luxurious hotel, casino and entertainment complex located in the heart of Singapore.

4. Universal Studios Singapore: Universal Studios Singapore is a theme park located on Sentosa Island. It features thrilling rides, shows and attractions based on popular films and television series.

5. Chinatown: Chinatown is a fascinating blend of old and new, where traditional Chinese architecture is juxtaposed with modern shopping malls and restaurants.

6. Gardens by the Bay: Gardens by the Bay is a sprawling 101-hectare park located in Marina Bay. It features two colossal cons

In [9]:
from langchain.chains import SimpleSequentialChain
from langchain import HuggingFaceHub

In [14]:
prompt_template = PromptTemplate(
    input_variables=["user_prompt"],
    template ="""You have to suggest 5 best places to visit in {user_prompt}"""
)

In [15]:
HF_llm= HuggingFaceHub(repo_id = "google/flan-t5-large")

/workspaces/MCQ-Creator-App/.venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [17]:
place_chain = LLMChain(llm=HF_llm, prompt=prompt_template)

In [31]:
place_chain.run(user_prompt="Singapore")

'Singapore Museum of Nature, Singapore Zoo, Singapore Botanical Gardens, Singapore Botanical Gardens'

In [19]:
template = """Given a list a places, please estimate the expenses to visit all of them in local currency {expenses}"""
prompt_template = PromptTemplate(
    input_variables=["expenses"],
    template = template
)

In [25]:
llm= OpenAI()
expenses_chain = LLMChain(llm=llm, prompt=prompt_template)

In [26]:
template = """Given a list a places, please estimate the expenses to visit all of them in local currency and also the days needed
{expenses}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(
    input_variables=["expenses"],
    template=template)

In [29]:
expenses_chain = LLMChain(llm=llm, prompt=prompt_template)

In [34]:
final_chain =  SimpleSequentialChain(chains=[place_chain, expenses_chain],verbose= True)

In [36]:
final_chain.run("Singapore")



> Entering new SimpleSequentialChain chain...
Singapore Museum of Nature, Singapore Zoo, Singapore Botanical Gardens, Singapore Botanical Gardens
The estimated expenses to visit all of these places in Singapore dollars will be around SGD 55.00 and it should take around 2 days to visit them all.

> Finished chain.


'The estimated expenses to visit all of these places in Singapore dollars will be around SGD 55.00 and it should take around 2 days to visit them all.'

In [19]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

In [20]:
llm = OpenAI(temperature=0.9)

In [21]:
with open("sample.txt") as f:
    data = f.read()

In [22]:
data

'\nMain menu\n\nWikipediaThe Free Encyclopedia\nSearch Wikipedia\nSearch\nCreate account\nLog in\n\nPersonal tools\nContents hide\n(Top)\nEtymology\nHistory\nToggle History subsection\nTypes\nToggle Types subsection\nHardware\nToggle Hardware subsection\nSoftware\nToggle Software subsection\nNetworking and the Internet\nUnconventional computers\nFuture\nToggle Future subsection\nProfessions and organizations\nSee also\nNotes\nReferences\nSources\nExternal links\nComputer\n\nArticle\nTalk\nRead\nView source\nView history\n\nTools\nPage semi-protected\nFrom Wikipedia, the free encyclopedia\nFor other uses, see Computer (disambiguation).\nMan replacing one vacuum tube out of hundreds in early computer\nComputer room with multiple computer cabinets and operating panel\nSmartphone with rainbow-like display held in a hand\nBlack desktop computer with monitor on top and keyboard in front\nPurple video game console with attached controller\nRows of large, dark computer cabinets in warehouse-li

In [23]:
text_splitter = CharacterTextSplitter()
texts = text_splitter.split_text(data)

Created a chunk of size 31222, which is longer than the specified 4000


In [25]:
len(texts)

22

In [26]:
docs = [Document(page_content=t) for t in texts]

In [28]:
len(docs)

22

In [31]:
chain= load_summarize_chain(llm=llm, chain_type="map_reduce")
chain.run(input_documents=docs)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 8591 tokens (8335 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}